1. Please download the original repository from Mauceri et al.
https://github.com/spaghettix/DB_OC_TSC
2. Put this code and the dataset to the same path as example.py.
3. Please select a dissimilarity metric.
4. Run.

In [ ]:
import os
import numpy as np
from prototype import prototype
import scipy.spatial.distance as ssd
from dissimilarity import dissimilarity
from sklearn.metrics import roc_curve, auc
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
import pickle
def makefile(what,filename):
    with open(filename,"wb") as f3:
        pickle.dump(what,f3)

def readfile(filename):
    with open(filename,"rb") as f4:
        ans=pickle.load(f4)
    return ans

# L=660

In [ ]:
%%time
result2=[]
for e in range(5):
    #dataset and class label.
    train=readfile("X.pickle")
    train_label=readfile("y.pickle")
    
    train, x_test, train_label, y_test = train_test_split(train, train_label, test_size=0.2, random_state=e)
    result=[]
    for i in range(20):
        x_train=train[train_label==i]
        y_label=np.zeros(len(y_test))
        y_label[y_test==i]=1
        D, P = dissimilarity(), prototype()
        
        #Please select a dissimilarity metric.
        Dissimilarity = D.kullback_leibler
        #Dissimilarity = D.autocorrelation
        #Dissimilarity = D.chebyshev
        #Dissimilarity = D.cityblock
        #Dissimilarity = D.cosine
        #Dissimilarity = D.euclidean
        #Dissimilarity = D.gaussian
        #Dissimilarity = D.sigmoid
        #Dissimilarity = D.wasserstein
        
        diss_params = {}

        Prot_method = P.centers_k_means



        # GET DISSIMILARITY MATRIX

        # Some prototype methods eg "centers_k_means" do not require
        # the computation of the dissimilarity matrix

        #Diss_Matrix = ssd.cdist(x_train, x_train, metric=Dissimilarity, **diss_params)


        # GET PROTOTYPES

        n = 2 # number of prototypes we want to get

        #Prototypes = Prot_method(x_train, n, Diss_Matrix)
        # OR
        Prototypes = Prot_method(x_train, n)



        # GET DISSIMILARITY-BASED REPRESENTATION

        DBR_X_train = ssd.cdist(x_train, Prototypes, Dissimilarity, **diss_params)
        DBR_X_test = ssd.cdist(x_test, Prototypes, Dissimilarity, **diss_params)

        # GET AUROC

        Classifier = NearestNeighbors(n_neighbors=1)
        Classifier.fit(DBR_X_train)

        Test_scores = Classifier.kneighbors(DBR_X_test)[0] * -1 
        # Test scores are multiplied by -1 because the ROC curve expects that 
        # more is better while in terms of dissimilarities less is better.

        fpr, tpr, _ = roc_curve(y_label, Test_scores, pos_label=1)
        AUROC = auc(fpr, tpr) * 100
        result.append(round(AUROC, 1))
        print('AUROC:', round(AUROC, 1))
    print(result)
    result2.append(result)
a=np.array(result2).mean(axis=0)
b=np.array(result2).std(axis=0)
for n in range(20):
    print(round(a[n],1),"±",round(b[n],1))
print(round(a.mean(),1))

# L=330, half signals

In [ ]:
%%time
result2=[]
for e in range(5):
    #dataset and class label.
    train=readfile("X.pickle")
    train_label=readfile("y.pickle")
    
    train, x_test, train_label, y_test = train_test_split(train, train_label, test_size=0.2, random_state=e)
    train=train.reshape(len(train)*2,330)
    train_label=np.concatenate([train_label.reshape(-1,1),train_label.reshape(-1,1)],axis=1).reshape(len(train_label)*2)
    x_test=x_test.reshape(len(x_test)*2,330)
    y_test=np.concatenate([y_test.reshape(-1,1),y_test.reshape(-1,1)],axis=1).reshape(len(y_test)*2)
    result=[]
    for i in range(20):
        x_train=train[train_label==i]
        y_label=np.zeros(len(y_test))
        y_label[y_test==i]=1
        D, P = dissimilarity(), prototype()
        
        #Please select a dissimilarity metric.
        Dissimilarity = D.kullback_leibler
        #Dissimilarity = D.autocorrelation
        #Dissimilarity = D.chebyshev
        #Dissimilarity = D.cityblock
        #Dissimilarity = D.cosine
        #Dissimilarity = D.euclidean
        #Dissimilarity = D.gaussian
        #Dissimilarity = D.sigmoid
        #Dissimilarity = D.wasserstein
        
        diss_params = {}

        Prot_method = P.centers_k_means



        # GET DISSIMILARITY MATRIX

        # Some prototype methods eg "centers_k_means" do not require
        # the computation of the dissimilarity matrix

        #Diss_Matrix = ssd.cdist(x_train, x_train, metric=Dissimilarity, **diss_params)


        # GET PROTOTYPES

        n = 2 # number of prototypes we want to get

        #Prototypes = Prot_method(x_train, n, Diss_Matrix)
        # OR
        Prototypes = Prot_method(x_train, n)



        # GET DISSIMILARITY-BASED REPRESENTATION

        DBR_X_train = ssd.cdist(x_train, Prototypes, Dissimilarity, **diss_params)
        DBR_X_test = ssd.cdist(x_test, Prototypes, Dissimilarity, **diss_params)

        # GET AUROC

        Classifier = NearestNeighbors(n_neighbors=1)
        Classifier.fit(DBR_X_train)

        Test_scores = Classifier.kneighbors(DBR_X_test)[0] * -1 
        # Test scores are multiplied by -1 because the ROC curve expects that 
        # more is better while in terms of dissimilarities less is better.

        fpr, tpr, _ = roc_curve(y_label, Test_scores, pos_label=1)
        AUROC = auc(fpr, tpr) * 100
        result.append(round(AUROC, 1))
        print('AUROC:', round(AUROC, 1))
    print(result)
    result2.append(result)
a=np.array(result2).mean(axis=0)
b=np.array(result2).std(axis=0)
for n in range(20):
    print(round(a[n],1),"±",round(b[n],1))
print(round(a.mean(),1))

# L=330, downsampling

In [ ]:
%%time
result2=[]
for e in range(5):
    #dataset and class label.
    train=readfile("X.pickle").reshape(len(train),330,2).mean(axis=2)
    train_label=readfile("y.pickle")
    
    train, x_test, train_label, y_test = train_test_split(train, train_label, test_size=0.2, random_state=e)
    result=[]
    for i in range(20):
        x_train=train[train_label==i]
        y_label=np.zeros(len(y_test))
        y_label[y_test==i]=1
        D, P = dissimilarity(), prototype()
        
        #Please select a dissimilarity metric.
        Dissimilarity = D.kullback_leibler
        #Dissimilarity = D.autocorrelation
        #Dissimilarity = D.chebyshev
        #Dissimilarity = D.cityblock
        #Dissimilarity = D.cosine
        #Dissimilarity = D.euclidean
        #Dissimilarity = D.gaussian
        #Dissimilarity = D.sigmoid
        #Dissimilarity = D.wasserstein
        
        diss_params = {}

        Prot_method = P.centers_k_means



        # GET DISSIMILARITY MATRIX

        # Some prototype methods eg "centers_k_means" do not require
        # the computation of the dissimilarity matrix

        #Diss_Matrix = ssd.cdist(x_train, x_train, metric=Dissimilarity, **diss_params)


        # GET PROTOTYPES

        n = 2 # number of prototypes we want to get

        #Prototypes = Prot_method(x_train, n, Diss_Matrix)
        # OR
        Prototypes = Prot_method(x_train, n)



        # GET DISSIMILARITY-BASED REPRESENTATION

        DBR_X_train = ssd.cdist(x_train, Prototypes, Dissimilarity, **diss_params)
        DBR_X_test = ssd.cdist(x_test, Prototypes, Dissimilarity, **diss_params)

        # GET AUROC

        Classifier = NearestNeighbors(n_neighbors=1)
        Classifier.fit(DBR_X_train)

        Test_scores = Classifier.kneighbors(DBR_X_test)[0] * -1 
        # Test scores are multiplied by -1 because the ROC curve expects that 
        # more is better while in terms of dissimilarities less is better.

        fpr, tpr, _ = roc_curve(y_label, Test_scores, pos_label=1)
        AUROC = auc(fpr, tpr) * 100
        result.append(round(AUROC, 1))
        print('AUROC:', round(AUROC, 1))
    print(result)
    result2.append(result)
a=np.array(result2).mean(axis=0)
b=np.array(result2).std(axis=0)
for n in range(20):
    print(round(a[n],1),"±",round(b[n],1))
print(round(a.mean(),1))